In [1]:
import torch
from llm.create import get_send_fn, get_text_emb_send_fn
from search.searcher import Searcher
from langchain import hub
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from lc_helper.wrapper import LcChatWrapper
from index.vectorstore import VectorStoreIndexer

print(torch.cuda.is_available())

USER_AGENT environment variable not set, consider setting it to identify your requests.


True


In [2]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [3]:
### Prameters to play with ###
llm_source = "openai"
model_name = "gpt-3.5-turbo"
vectorstore_dir= "./index_results/vectorstore/atelectasis_top_1_chatgpt3_5"


# llm_source = "huggingface"
# model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"
# vectorstore_dir= "./index_results/vectorstore/atelectasis_top_1_llama3_1"
##############################

In [4]:
# import json, os

In [5]:
# with open(os.path.join(vectorstore_dir, "emb_llm.json"), "w") as f:
#         spec_dict = json.load(f)

In [6]:
vectorstore = VectorStoreIndexer.load_db(vectorstore_dir)
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")
llm = LcChatWrapper(send_fn=get_send_fn(source=llm_source, model_name=model_name))
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [7]:
res = rag_chain.invoke("What are symptoms of atelectasis?")

In [8]:
print(res)

Symptoms of atelectasis can include cough, chest pain, difficulty breathing, low oxygen saturation, pleural effusion, cyanosis, and increased heart rate. It is a common misconception that atelectasis causes fever, with clinical evidence not supporting this speculation. Causes of atelectasis can range from post-surgical complications to blockages in the airways.


In [9]:
# prompt
print(llm.last_messages[-1]['content'])

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: What are symptoms of atelectasis? 
Context: atelectasis is the partial collapse or closure of a lung resulting in reduced or absent gas exchange. it is usually unilateral, affecting part or all of one lung. it is a condition where the alveoli are deflated down to little or no volume, as distinct from pulmonary consolidation, in which they are filled with liquid. it is often referred to informally as a collapsed lung, although more accurately it usually involves only a partial collapse, and that ambiguous term is also informally used for a fully collapsed lung caused by a pneumothorax.
it is a very common finding in chest x-rays and other radiological studies, and may be caused by normal exhalation or by various medical conditions. although fr

In [10]:
# chatgpt3.5
# Symptoms of atelectasis may include cough, chest pain (though not common), breathing difficulty, low oxygen saturation, pleural effusion, cyanosis (late sign), and increased heart rate. It is a common misconception that atelectasis causes fever, as studies have shown no clinical evidence supporting this speculation. The most common causes of atelectasis are post-surgical atelectasis after abdominal surgery and pulmonary tuberculosis, with risk factors including type of surgery, use of muscle relaxation, obesity, age, and presence of chronic obstructive pulmonary disease or asthma.

# Llama3.1
# Symptoms of atelectasis may include breathlessness, cough, and pleuritic chest pain. Atelectasis can also lead to reduced oxygen levels in the blood, which may cause symptoms such as fatigue and confusion. In severe cases, it may cause severe respiratory distress.